In [22]:
import pykitti
import numpy as np
import open3d as o3d

# 坐标转换

In [23]:
basedir = '../datasets'
date = '2011_10_03'
drive = '0027'
dataset = pykitti.raw(basedir, date, drive)
data = dataset.get_velo(0)
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(data[:, :3])

绕坐标轴旋转

In [5]:
from scipy.spatial.transform import Rotation as R
R1 = R.from_euler('y', 90, degrees=True).as_matrix()
R2 = R.from_euler('x', -90, degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R1, center=point_cloud.get_center()).rotate(R2, center=point_cloud.get_center())])
# 复合旋转
R3 = R.from_euler('zyx', [45, 30, 60], degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R3, center=point_cloud.get_center())])

使用四元数、旋转向量旋转

In [6]:
R4 = R.from_quat([0.5, 0.5, 0.5, 0.5])
data4 = np.dot(data[:, :3], R4.as_matrix())
point_cloud4 = o3d.geometry.PointCloud()
point_cloud4.points = o3d.utility.Vector3dVector(data4)
o3d.visualization.draw_geometries([point_cloud4])

R5 = R.from_rotvec([0.1, 0.2, 0.3])
data5 = np.dot(data[:, :3], R5.as_matrix())
point_cloud5 = o3d.geometry.PointCloud()
point_cloud5.points = o3d.utility.Vector3dVector(data5)
o3d.visualization.draw_geometries([point_cloud5])

# 将GT坐标 from cam to velo

读取GT坐标、calib文件

In [7]:
def read_calib_file(filepath):
    calib_data = {}
    with open(filepath, 'r') as file:
        for line in file:
            key, value = line.split(':', 1)
            if key == 'R' or key == 'T':
                calib_data[key] = value.strip()

    return calib_data


filepath = '../datasets/2011_10_03/calib_velo_to_cam.txt'
calib_data = read_calib_file(filepath)

# 解析和转换 R 和 T 数据
if "R" in calib_data and "T" in calib_data:
    R_values = [float(x) for x in calib_data["R"].split()]
    T_values = [float(x) for x in calib_data["T"].split()]

    # 构造变换矩阵
    R_matrix = np.array(R_values).reshape(3, 3)
    T_vector = np.array(T_values).reshape(3, 1)
    RT_matrix = np.hstack((R_matrix, T_vector))
    RT_matrix = np.vstack((RT_matrix, [0, 0, 0, 1]))

    RT_matrix
else:
    RT_matrix = None

RT_matrix



array([[ 7.967514e-03, -9.999679e-01, -8.462264e-04, -1.377769e-02],
       [-2.771053e-03,  8.241710e-04, -9.999958e-01, -5.542117e-02],
       [ 9.999644e-01,  7.969825e-03, -2.764397e-03, -2.918589e-01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

对GT坐标进行变换

In [8]:
RT_matrix_inv = np.linalg.inv(RT_matrix)

gt_filepath = '../datasets/2011_10_03/2011_10_03_drive_0027_sync/00.txt'

# 读取轨迹数据
with open(gt_filepath, 'r') as f:
    lines = f.readlines()

transformed_lines = []

for line in lines:
    pose = line.strip().split()  # 切分每行数据

     # 提取位姿变换矩阵并变为4x4
    pose_matrix = np.eye(4)
    
    pose_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)

    # 旋转矩阵
    transformed_matrix = RT_matrix_inv @ pose_matrix @ RT_matrix

    # 从变换矩阵中提取变换后的位姿
    transformed_pose = transformed_matrix[:3, :].flatten()
    transformed_lines.append(" ".join(map(str, transformed_pose)))

# 保存变换后的轨迹
save_filepath = '../datasets/2011_10_03/2011_10_03_drive_0027_sync/groundtruth.txt'

with open(save_filepath, 'w') as f:
    f.write("\n".join(transformed_lines))


In [9]:
import numpy as np
import pykitti
import open3d as o3d

# GroundTruth cam to velo -> velo to velo

In [11]:
# 读取groundtruth
file_path = '../datasets/2011_10_03/2011_10_03_drive_0027_sync/groundtruth.txt'
with open(file_path,'r') as file:
    data = file.readlines()

data_matrices = []  # 用于存储所有的4x4矩阵
for line in data:
    pose = line.strip().split()  # 切分每行数据
    data_matrix = np.eye(4)
    data_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)
    data_matrices.append(data_matrix)

data_matrices = np.array(data_matrices)

print(data_matrices.shape)
# print(data_matrices[5])



# transformation_matrix = compute_transformation_matrix(data[5], data[10])
# print(transformation_matrix) # data[5]映射到data[10]上

(4541, 4, 4)


In [12]:
import copy
# o3d可视化点云配准结果
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [13]:
def get_pointcloud(x,y):
    #取第x帧和第y帧的点云
    data1 = dataset.get_velo(x)
    data2 = dataset.get_velo(y)
    point_cloud1 = o3d.geometry.PointCloud()
    data1 = data1[:, :3]
    point_cloud1.points = o3d.utility.Vector3dVector(data1)
    point_cloud2 = o3d.geometry.PointCloud()
    data2 = data2[:, :3]
    point_cloud2.points = o3d.utility.Vector3dVector(data2)
    T = np.dot(np.linalg.inv(data_matrices[x]), data_matrices[y])
    print(T)
    return point_cloud1, point_cloud2, T

point_cloud1, point_cloud2, T = get_pointcloud(25, 30)

#变换矩阵
draw_registration_result(point_cloud2, point_cloud1, T)

[[ 9.99949417e-01  4.79809322e-03  8.84326530e-03  4.71342125e+00]
 [-4.80965672e-03  9.99987595e-01  1.28602157e-03  8.46553248e-02]
 [-8.83698657e-03 -1.32848972e-03  9.99960088e-01  4.11747342e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [21]:
# 获取真值转换的所有T
T_all = []
print(data_matrices.shape[0])
for i in range(data_matrices.shape[0] - 1):
    T = np.dot(np.linalg.inv(data_matrices[i+1]), data_matrices[i])
    T_all.append(T)

T_all = np.array(T_all)

print(T_all.shape)
print(T_all[0])

4541
(4540, 4, 4)
[[ 9.99997186e-01  2.06740258e-03  1.16075893e-03 -8.58417869e-01]
 [-2.06680528e-03  9.99997657e-01 -5.14705654e-04 -5.13599191e-02]
 [-1.16181991e-03  5.12305062e-04  9.99999187e-01 -2.47504895e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
